In [ ]:
import json
import pandas as pd
import requests
import geopy
from geopy import distance
from geopy.distance import geodesic

# Load json object downlaoded from citybikes api

## Investigate the structure of the returned data using .keys()
Return the DataFrame that captures the data: empty_slots, free bikes, lat, long, and name.

In [ ]:
with open('citybikes-helsinki.json') as f:
    bikes = json.load(f)

bikes = pd.DataFrame(bikes['network']['stations'])
bikes

### check out some of the characteristics of the returned data

In [ ]:
bikes.describe()

This has returned too many results, I won't be able to query this many stations through the yelp api.
I will reduce these stations to a radius of 3.5km around the center of Helsinki using lat/long values.
According to NASA, helsinki center = lat,long(60.1711,24.9326).

#### Lets test this formula, as this isn't how it's shown in the documentation, it gets a good result.

In [ ]:
geopy.distance.distance((60.1711,24.9326), (bikes["latitude"][0],bikes["longitude"][0])).km

### Let's try creating a new variable and populating it with these created values. 
BUT, the solution below only calculates one value.

In [ ]:
bikes["test"] = (

    geopy.distance.distance((60.1711,24.9326),((bikes["latitude"][0]),(bikes["longitude"][0]))).km

)
bikes

This will calculate along all the rows in the dataset

In [ ]:
def calculate_distance(row):
    coordinate_1 = (60.1711, 24.9326)
    coordinate_2 = (row['latitude'], row['longitude'])
    return geodesic(coordinate_1, coordinate_2).km

#### And let's now create that new variable

In [ ]:
bikes["distance_from_center"] = bikes.apply(calculate_distance, axis = 1)

#### Now, let's narrow down the stations to those within 3.5km from the center point of Helsinki
We've narrowed 583 stations down to a respectable 123. 

In [ ]:
bikes_center = pd.DataFrame(bikes.loc[(bikes['distance_from_center'] <= 3.5)])

## Let's check and see whether this worked!

In [ ]:
Bikes_center.sort_values('distance_from_center').tail(25)

Okay, now that this baby is almost ready to work with, lets clean it up a little. I'll save working with the date, because they're all the same date. I'll keep it for archival, but there's no need to work with it here. 
Let's drop things we aren't going to use.
We're dropping 'extra' = (iud:number), and 'test' = I created, and 'lat_long' = I created.

In [ ]:
bikes_center.drop(columns='extra',inplace=True)
bikes_center.drop(columns='test',inplace=True)
bikes_center.drop(columns='lat_long',inplace=True)

Let's create a new column for the merge to the yelp_restaurants data.

In [ ]:
bikes_center['mergevar'] = 1

Now, lets save this table as a csv... just in case.

In [ ]:
bikes_center.to_csv(r'C:\Users\ca0pa\Desktop\LHL\Python\bikes_center.csv', index=False)